# XPS WorkChain

Here is an example of how to use the `XpsWorkChain` to compute the XPS spectrum.


First, we import the core-hole pseudopotential, run the following command in the terminal:

```console
verdi archive import https://github.com/superstar54/xps-data/raw/main/pseudo_demo/pseudo_demo_pbe.aiida
```

## Extended system


In [1]:
from aiida import load_profile, orm
from ase.build import bulk
from aiida.engine import run_get_node
from aiida_qe_xspec.workflows.xps import XpsWorkChain
from aiida_qe_xspec.utils import load_core_hole_pseudos
import numpy as np
from aiida import orm


load_profile()

atoms = bulk("Si")
structure = orm.StructureData(ase=atoms)
code = orm.load_code("qe-7.2-pw@localhost")
# Load the pseudopotential family.
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([5, 5, 5])
#
options = {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 4,
        },
    }
structure_preparation_settings = {
    'supercell_min_parameter': orm.Float(1.0),
    'is_molecule_input': orm.Bool(False),
}
# Load the pseudopotential family.
core_levels = {"Si": ["2p"]}
core_hole_pseudos, correction_energies = load_core_hole_pseudos(core_levels, "pseudo_demo_pbe")
core_hole_treatments={"Si": "xch_smear"}
builder = XpsWorkChain.get_builder_from_protocol(
    structure=structure,
    code=code,
    protocol="fast",
    overrides = {"ch_scf": {"pseudo_family": "SSSP/1.2/PBE/efficiency"}},
    core_hole_pseudos=core_hole_pseudos,
    core_levels=core_levels,
    calc_binding_energy=orm.Bool(True),
    correction_energies=orm.Dict(correction_energies),
    core_hole_treatments=core_hole_treatments,
    structure_preparation_settings=structure_preparation_settings,
    kpoints=kpoints,
    options=options,
)
builder.pop("relax")
_, node = run_get_node(builder)

print("Binding energy of Si 2p core level is")
print(node.outputs.binding_energies.get_dict())



02/07/2025 10:49:08 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23660|XpsWorkChain|prepare_structures]: structures_to_process: {'site_0': <StructureData: uuid: d59245da-24a2-4a4c-ae3a-af066f1b7a24 (pk: 23665)>}
02/07/2025 10:49:08 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23660|XpsWorkChain|run_gs_scf]: launched PwBaseWorkChain for supercell<23667>


site site_0
abs_element Si


02/07/2025 10:49:09 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23660|XpsWorkChain|run_all_scf]: launched PwBaseWorkChain for Si_site_0_2p<23669>
02/07/2025 10:49:10 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23669|PwBaseWorkChain|run_process]: launching PwCalculation<23676> iteration #1
02/07/2025 10:49:11 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23667|PwBaseWorkChain|run_process]: launching PwCalculation<23684> iteration #1
02/07/2025 10:49:12 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23669|PwBaseWorkChain|sanity_check_insufficient_bands]: PwCalculation<23676> run with smearing and highest band is occupied
02/07/2025 10:49:12 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23669|PwBaseWorkChain|sanity_check_insufficient_bands]: BandsData<23679> has invalid occupations: Occupation of 1.000932470972213 at l

result {'chemical_shifts': <Dict: uuid: f26a4a54-d570-43d2-ad00-90e8c033da55 (pk: 23701)>, 'chemical_shift_spectra': {'Si_2p': <XyData: uuid: edd1c5ec-d075-4921-ba97-4cdb949b94e2 (pk: 23702)>}, 'binding_energy_spectra': {'Si_2p': <XyData: uuid: 29557140-8791-4fa7-82c3-682963472139 (pk: 23703)>}, 'binding_energies': <Dict: uuid: 4a6cb923-d398-4b79-abdd-6c3aa52d2c3f (pk: 23704)>}
Binding energy of Si 2p core level is
{'Si': {'2p': {'site_0': {'energy': 99.8438337976, 'multiplicity': 8}}}}


## Molecular system



In [2]:
from aiida import load_profile, orm
from ase.build import molecule
from aiida.engine import run_get_node
from aiida_qe_xspec.workflows.xps import XpsWorkChain
from aiida_qe_xspec.utils import load_core_hole_pseudos
from aiida import orm


load_profile()

atoms = molecule('H2O')
atoms.center(vacuum=3.0)
atoms.pbc = True
structure = orm.StructureData(ase=atoms)
code = orm.load_code("qe-7.2-pw@localhost")
# Load the pseudopotential family.
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])
#
options = {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 4,
        },
    }
structure_preparation_settings = {
    'supercell_min_parameter': orm.Float(4.0),
    'is_molecule_input': orm.Bool(True),
}
# Load the pseudopotential family.
core_levels = {"O": ["1s"]}
core_hole_pseudos, correction_energies = load_core_hole_pseudos(core_levels, "pseudo_demo_pbe")
core_hole_treatments={"O": "full"}
builder = XpsWorkChain.get_builder_from_protocol(
    structure=structure,
    code=code,
    protocol="fast",
    overrides = {"ch_scf": {"pseudo_family": "SSSP/1.2/PBE/efficiency"}},
    core_hole_pseudos=core_hole_pseudos,
    core_levels=core_levels,
    calc_binding_energy=orm.Bool(True),
    correction_energies=orm.Dict(correction_energies),
    core_hole_treatments=core_hole_treatments,
    structure_preparation_settings=structure_preparation_settings,
    kpoints=kpoints,
    options=options,
)
builder.pop("relax")
_, node = run_get_node(builder)

print("Binding energy of O 1s core level is")
print(node.outputs.binding_energies.get_dict())



02/07/2025 10:49:16 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23723|XpsWorkChain|prepare_structures]: structures_to_process: {'site_0': <StructureData: uuid: e1b32f69-a97f-4810-b48b-7ed3140c4498 (pk: 23727)>}
02/07/2025 10:49:17 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23723|XpsWorkChain|run_gs_scf]: launched PwBaseWorkChain for supercell<23729>


site site_0
abs_element O


02/07/2025 10:49:17 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23723|XpsWorkChain|run_all_scf]: launched PwBaseWorkChain for O_site_0_1s<23731>
02/07/2025 10:49:18 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23729|PwBaseWorkChain|run_process]: launching PwCalculation<23734> iteration #1
02/07/2025 10:49:19 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23731|PwBaseWorkChain|run_process]: launching PwCalculation<23742> iteration #1
02/07/2025 10:49:20 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23729|PwBaseWorkChain|results]: work chain completed after 1 iterations
02/07/2025 10:49:20 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23729|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned
02/07/2025 10:49:20 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23731|PwBaseWo

result {'chemical_shifts': <Dict: uuid: 6b67ddb5-336a-429b-98d6-ae2b20d69a4f (pk: 23751)>, 'chemical_shift_spectra': {'O_1s': <XyData: uuid: 07a1dace-2079-4883-bb96-ba20f1ebfa62 (pk: 23752)>}, 'binding_energy_spectra': {'O_1s': <XyData: uuid: 9249650c-7923-44d0-b52f-5559cab4df92 (pk: 23753)>}, 'binding_energies': <Dict: uuid: 49363e69-fb9d-4706-afbb-8a3d65013ec0 (pk: 23754)>}
Binding energy of O 1s core level is
{'O': {'1s': {'site_0': {'energy': 538.99493145453, 'multiplicity': 1}}}}


## Run calculation for selected atoms
The previous examples calculated the XPS spectra for selected elements by analyzing the symmetry and finding all non-equivalent sites. This is not suitable for large systems with low symmetry, e.g. supported nanoparticles, in which all atoms are non-equivalent and the user is usually only interested in the spectra of some special atoms.

You can use the `atom_indices` input to specify the atoms you are interested in.

In [3]:
from aiida import load_profile, orm
from ase.build import molecule
from aiida.engine import run_get_node
from aiida_qe_xspec.workflows.xps import XpsWorkChain
from aiida_qe_xspec.utils import load_core_hole_pseudos
from aiida import orm

load_profile()

atoms = molecule('CH3CHO')
atoms.center(vacuum=3.0)
atoms.pbc = True
structure = orm.StructureData(ase=atoms)
code = orm.load_code("qe-7.2-pw@localhost")
# Load the pseudopotential family.
kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])
#
options = {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 4,
        },
    }
structure_preparation_settings = {
    'supercell_min_parameter': orm.Float(4.0),
    'is_molecule_input': orm.Bool(True),
}
# Load the pseudopotential family.
core_levels = {"C": ["1s"]}
core_hole_pseudos, correction_energies = load_core_hole_pseudos(core_levels, "pseudo_demo_pbe")
core_hole_treatments={"C": "full"}
builder = XpsWorkChain.get_builder_from_protocol(
    structure=structure,
    code=code,
    protocol="fast",
    overrides = {"ch_scf": {"pseudo_family": "SSSP/1.2/PBE/efficiency"}},
    core_hole_pseudos=core_hole_pseudos,
    core_levels=core_levels,
    atom_indices=[1],
    calc_binding_energy=orm.Bool(True),
    correction_energies=orm.Dict(correction_energies),
    core_hole_treatments=core_hole_treatments,
    structure_preparation_settings=structure_preparation_settings,
    kpoints=kpoints,
    options=options,
)
builder.pop("relax")
_, node = run_get_node(builder)

print("Binding energy of C 1s core level is")
print(node.outputs.binding_energies.get_dict())



02/07/2025 10:55:41 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23946|XpsWorkChain|prepare_structures]: structures_to_process: {'site_1': <StructureData: uuid: 8af5f8e4-1b41-4564-9e3b-34f92c2c5f9d (pk: 23949)>}
02/07/2025 10:55:42 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23946|XpsWorkChain|run_gs_scf]: launched PwBaseWorkChain for supercell<23951>


site site_1
abs_element C


02/07/2025 10:55:42 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23946|XpsWorkChain|run_all_scf]: launched PwBaseWorkChain for C_site_1_1s<23953>
02/07/2025 10:55:43 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23951|PwBaseWorkChain|run_process]: launching PwCalculation<23956> iteration #1
02/07/2025 10:55:44 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23953|PwBaseWorkChain|run_process]: launching PwCalculation<23964> iteration #1
02/07/2025 10:55:45 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23951|PwBaseWorkChain|results]: work chain completed after 1 iterations
02/07/2025 10:55:45 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23951|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned
02/07/2025 10:55:45 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23953|PwBaseWo

result {'chemical_shifts': <Dict: uuid: 141d4599-c5ee-4565-a51d-ac3e4cec6166 (pk: 23973)>, 'chemical_shift_spectra': {'C_1s': <XyData: uuid: f96475ff-5421-4c4f-b514-46b67531cc20 (pk: 23974)>}, 'binding_energy_spectra': {'C_1s': <XyData: uuid: 427b6855-2cf8-423e-83ae-683ba154d7c7 (pk: 23975)>}, 'binding_energies': <Dict: uuid: 81451498-49b2-47bb-b20f-d92d0e2f24ba (pk: 23976)>}


02/07/2025 10:55:46 PM <606530> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [23946|XpsWorkChain|on_terminated]: remote folders will not be cleaned


Binding energy of C 1s core level is
{'C': {'1s': {'site_1': {'energy': 293.8658379553, 'multiplicity': 1}}}}
